## Training on sagemaker with base pytorch image

Here all necessary libraries, variables to start the training.
This scenario covers training with default pytorch image, on GPU (1 instance). Training code is in "code" directory.
!!! NOTE : for VPC training, some additiona settings should be set + we will need a different image, with proxy set. 

In [ ]:
import pandas as pd
import sagemaker
import time

#get sagemaker session, role and region
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name

bucket = "yourbucker" 
kms_key = "kmskey"
output_path = "s3://{}/out".format(bucket)
module_dir="s3://{}/module".format(bucket)
training_data = "s3//pathtoyourdata"
role

###### What below code will do:
1. Start an estimator object - an abstraction level class for Pytorch framework
2. Pack your code (all content of code directory) and put it to "module_dir" s3 path
3. Launch a training job in pipe mode
4. In code file, you can specify requirements.txt with python modules to install. Pytorch container will do it for you

###### A note on Pipe mode with Pytorch:
1. It reads bytes from the file you set as training file. In train.py you specify how many bytes are to be read. 
2. There is a helper created by me for transforming bytes to dataframe (since we have bytes read from object, it might happen it reads half of the row, or half of the object in the row). It is not perfect (it drops cases he cannot decode), but it helps with the transformation. 
3. Data will be read sequencially, until the EOF, and will start again on next epoch. Each sequence will be transformed in dataframe (by train.py script) and will be fed for training. 

In [ ]:
from sagemaker.pytorch import PyTorch
from sagemaker.estimator import Estimator

estimator = PyTorch(
    source_dir='code',
    entry_point='train.py',
    code_location=module_dir,
    output_path=output_path,
    framework_version="1.6.0",
    py_version="py3",
    output_kms_key=kms_key,
    role=role,
    instance_count = 1,
    instance_type='ml.p2.xlarge',
    input_mode='Pipe'
)
estimator.fit(training_data)